In [1]:
import pandas as pd
import altair as alt
import numpy as np

# Initial Beta Only

In [25]:
import numpy as np

def run_sir(beta, gamma, population, num_infected, N):
    i_n = num_infected
    s_n = population - i_n
    r_n = 0.
    inew_n = 1
    
    s_a, i_a, r_a, inew_a = np.zeros(N), np.zeros(N), np.zeros(N), np.zeros(N)
    
    for index in range(N):
        s_a[index] = s_n
        i_a[index] = i_n
        r_a[index] = r_n
        inew_a[index] = inew_n

        s = s_n
        i = i_n
        r = r_n
        inew = inew_n
        
        s_n = (-beta * s * i) + s
        i_n = (beta * s * i - gamma * i) + i
        r_n = gamma * i + r
        inew_n = (beta * s * i)
    return s_a, i_a, r_a, inew_a

In [26]:
infectious_days = 14.
doubling_rate = 5
population = 1000000.
num_infected = 100.
N = 250

gamma = 1 / infectious_days
growth_rate = 2 ** (1 / doubling_rate) - 1
beta = (growth_rate + gamma) / (population - 1)

s, i, r, i_new = run_sir(beta, gamma, population, num_infected, N)

In [27]:
d = pd.DataFrame({
    'day': np.arange(len(s)), 
    'susceptable': s, 
    "infected": i, 
    "recovered": r, 
    "new_infections": i_new, 
    "infections_cumulative": np.cumsum(i_new),
    "penn_ever_infected": i + r,
})

(alt
 .Chart(d)
 .transform_fold(['susceptable', 'infected', 'recovered', 'new_infections', 'infections_cumulative'])
 .encode(x='day:Q', y="value:Q", color='key:N')
 .mark_line()
)

alt.Chart(...)

In [28]:
def doubling_times(i):
    i_cumulative = np.cumsum(i)
    delta = np.diff(i_cumulative, 1)
    growth_rate = delta / i_cumulative[1:]
    return 1 / np.log2(growth_rate + 1)

In [29]:
d_times = doubling_times(i_new)
rate = np.diff(d_times, 1)
d_time = pd.DataFrame({
    'day': np.arange(len(s) - 2), 
    'doubling_time': d_times[:-1],
    'rate_of_increase': rate,
})

(alt
 .Chart(d_time.iloc[:100])
 .encode(x='day:Q', y='doubling_time:Q')
 .mark_line()
)

alt.Chart(...)

This plot shows that doubling time changes with time. No social mitigation is required. It will change differently if there is social mitigation. We need to be able to distinguish the change in doubling time due to each.

In [30]:
(alt
 .Chart(d_time.iloc[:40])
 .encode(x='day:Q', y='rate_of_increase:Q')
 .mark_line()
)

alt.Chart(...)

# Variable Beta

In [31]:
import numpy as np

def run_sir(beta, gamma, population, num_infected, N):
    i_n = num_infected
    s_n = population - i_n
    r_n = 0.
    inew_n = 1.
    
    s_a, i_a, r_a, inew_a = np.zeros(N), np.zeros(N), np.zeros(N), np.zeros(N)
    
    for index in range(N):
        s_a[index] = s_n
        i_a[index] = i_n
        r_a[index] = r_n
        inew_a[index] = inew_n

        s = s_n
        i = i_n
        r = r_n
        inew = inew_n
        
        s_n = (-beta[index] * s * i) + s
        i_n = (beta[index] * s * i - gamma * i) + i
        r_n = gamma * i + r
        inew_n = (beta[index] * s * i)
    return s_a, i_a, r_a, inew_a

In [32]:
# Use variable doubling times
growth_rate = 2 ** (1 / d_times) - 1
beta_var = (growth_rate + gamma) / (population - 1)
s, i, r, i_new = run_sir(beta_var, gamma, population, num_infected, N - 1)

In [33]:
d = pd.DataFrame({
    'day': np.arange(len(s)), 
    'susceptable': s, 
    "infected": i, 
    "recovered": r, 
    "new_infections": i_new, 
    "infections_cumulative": np.cumsum(i_new),
    "penn_ever_infected": i + r,
})

(alt
 .Chart(d)
 .transform_fold(['susceptable', 'infected', 'recovered', 'new_infections', 'infections_cumulative'])
 .encode(x='day:Q', y="value:Q", color='key:N')
 .mark_line()
)

alt.Chart(...)

In [43]:
d_beta = pd.DataFrame({
    "day":d.day,
    "beta": beta,
    "beta_var": beta_var
})
(alt
 .Chart(d_beta[:])
 .transform_fold(["beta", "beta_var"])
 .encode(x='day:Q', y="value:Q", color='key:N')
 .mark_line()
)

alt.Chart(...)

Beta calculated using the changing doubling time. Beta describes contact rate between individuals.

# Try to Calculate Beta from Data

In [46]:
# la, nyc, slc county data from nytimes county level data
la = pd.read_csv("la_data.csv")
nyc = pd.read_csv("la_data.csv")
slc = pd.read_csv("la_data.csv")


In [47]:
# add admits to data for LA county.
la["admits"] = la.cases - la.cases.shift()
# Pretend customer gave us rows 60-65
la_train = la.iloc[0:]
la_train = la_train[["date", "cases", "admits"]]
# Use rows 66-end to project
la_test = la.iloc[66:]
la_test = la_test[["date", "cases", "admits"]]

In [48]:
gamma = 1/14
population = 10000000
la_train["growth_rate"] = la_train.admits / la_train.cases
la_train["beta"] = (la_train.growth_rate + gamma) / population
la_train["constant_beta"] = 2.2e-7 
la_train

,date,cases,admits,growth_rate,beta,constant_beta
0,2020-01-26,1,NaN,NaN,NaN,2.200000e-07
1,2020-01-27,1,0.0,0.000000,7.142857e-09,2.200000e-07
2,2020-01-28,1,0.0,0.000000,7.142857e-09,2.200000e-07
3,2020-01-29,1,0.0,0.000000,7.142857e-09,2.200000e-07
4,2020-01-30,1,0.0,0.000000,7.142857e-09,2.200000e-07
...,...,...,...,...,...,...
70,2020-04-05,5940,663.0,0.111616,1.830447e-08,2.200000e-07
71,2020-04-06,6360,420.0,0.066038,1.374663e-08,2.200000e-07
72,2020-04-07,6910,550.0,0.079595,1.510234e-08,2.200000e-07
73,2020-04-08,7530,620.0,0.082337,1.537659e-08,2.200000e-07


In [49]:

(alt
 .Chart(la_train.loc[la_train.cases > 10])
 .transform_fold(["beta", "constant_beta"])
 .encode(x='date:T', y="value:Q", color='key:N')
 .mark_line()
)

alt.Chart(...)

Beta vs. LA county data. Beta is calculated based on the growth rate, which is only valid in at time 0.

# Beta vs. initial doubling time

In [50]:
infectious_days = 14.
doubling_rate = np.arange(3, 10, .1)
population = 1000000.
num_infected = 10000.
N = 250

gamma = 1 / infectious_days
growth_rate = 2 ** (1 / doubling_rate) - 1
beta = (growth_rate + gamma) / (population - 1)
beta
df2 = pd.DataFrame({
    "initial_doubling": doubling_rate,
    "beta": beta,
    "real_beta": 2e-7,
})

(alt
 .Chart(df2)
 .transform_fold(["beta", "real_beta"])
 .encode(x='initial_doubling:Q', y="value:Q", color='key:N')
 .mark_line()
)

alt.Chart(...)

# Problem List:
We get data on number of cases and:
* could calculate initial beta and doubling time, as long as no social mitigation was in place.
* could calculate effect of social mitigation if onset and initial doubling time was known.

Either need: 
* **User to supply us with a reliable onset and initial doubling time**
* Or a way to estimate beta without using doubling time/growth rate
* Or need a way to differentiate increase in doubling time due to immunity (time) vs. mitigation.